In [23]:
from pydantic import BaseModel
from typing import List


class Info(BaseModel):
    """Data model for a Information."""

    flightNumber: str
    question: List[str]

In [13]:
from llama_index.multi_modal_llms.replicate import ReplicateMultiModal
from llama_index.core.program import MultiModalLLMCompletionProgram
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.multi_modal_llms.replicate.base import (
    REPLICATE_MULTI_MODAL_LLM_MODELS,
)

prompt_template_str = """\
    can you summarize what is in the image\
    and return the answer with json format \
"""


def pydantic_llava(
    model_name, output_class, image_documents, prompt_template_str
):
    mm_llm = ReplicateMultiModal(
        model=REPLICATE_MULTI_MODAL_LLM_MODELS["llava-13b"],
        max_new_tokens=1000,
    )

    llm_program = MultiModalLLMCompletionProgram.from_defaults(
        output_parser=PydanticOutputParser(output_class),
        image_documents=image_documents,
        prompt_template_str=prompt_template_str,
        multi_modal_llm=mm_llm,
        verbose=True,
    )

    response = llm_program()
    # print(f"Model: {model_name}")
    for res in response:
        print(res)
    return response

In [14]:
from llama_index.core import SimpleDirectoryReader

ins_image_documents = SimpleDirectoryReader("./pics").load_data()

In [15]:
import nest_asyncio
nest_asyncio.apply()

In [35]:
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_hub.tools.tavily_research import TavilyToolSpec
from llama_parse import LlamaParse
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent import AgentRunner
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent, ReActAgentWorker
from crawler import get_questions
from prompts import assistant_prompt, translator_prompt
from helper_functions import (fetch_user_flight_information, 
                              search_flights, 
                              cancel_ticket, 
                              update_ticket_to_new_flight,
                              book_hotel,
                              cancel_hotel,
                              update_hotel,
                              search_hotels,
                              book_car_rental,
                              cancel_car_rental,
                              update_car_rental,
                              search_car_rentals,
                              book_excursion,
                              search_trip_recommendations,
                              update_excursion,
                              cancel_excursion
                            )
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
from dotenv import load_dotenv
import os
import ast
from create_database import db


from llama_index.embeddings.fastembed import FastEmbedEmbedding

load_dotenv()

llm = Ollama(base_url='https://af17-34-125-107-68.ngrok-free.app', model="llama3", request_timeout=360)
# llm = Ollama(base_url='http://localhost:11434', model="llama3", request_timeout=360)

parser = LlamaParse(result_type="markdown")

file_extractor = {".docx": parser}

documents = SimpleDirectoryReader("./data").load_data()

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="BAAI/bge-small-en-v1.5"
# )

print('embedding')
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
print('embedded')
get_questions()

if os.path.exists('./storage'):
    storage_context = StorageContext.from_defaults(persist_dir='./storage')
    vector_index = load_index_from_storage(storage_context=storage_context, embed_model=embed_model)
else:    
    vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)    
    vector_index.storage_context.persist()

chat_text_qa_msgs = [
    (
        "user",
        """You are a Q&A assistant. Your goal is to answer questions as
        accurately as possible based on the instructions and context provided.
         
        Context:
         
        {assistant_prompt}
         
        Question:
         
        {prompt}
        """,
    )
]
# text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)
query_engine = vector_index.as_query_engine(llm=llm, similarity_top_k=1, num_thread=8, temperature=0)#, text_qa_template=text_qa_template)

lookup_policy = QueryEngineTool(
                        query_engine=query_engine,
                        metadata=ToolMetadata(
                            description="""Consult the company policies to check whether certain options are permitted.
                                            Use this before making any flight changes performing other 'write' events.""",
                            name='policy_documentation'
                        )
                    )

tavily_tool = TavilyToolSpec(
    api_key='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',
)

tools = [
    tavily_tool.to_tool_list()[0],
    lookup_policy,
    fetch_user_flight_information,
    search_flights,
    update_ticket_to_new_flight,
    cancel_ticket,
    book_hotel,
    cancel_hotel,
    update_hotel,
    search_hotels,
    book_car_rental,
    cancel_car_rental,
    update_car_rental,
    search_car_rentals,
    book_excursion,
    search_trip_recommendations,
    update_excursion,
    cancel_excursion
]

tools_descs = [tol.metadata.name for tol in tools]

customer_agent = ReActAgent.from_tools(tools, llm=llm, verbose=False, context=assistant_prompt, num_thread=8)
# customer_agent = ReActAgentWorker.from_tools(tools, llm=llm, verbose=True, context=assistant_prompt, num_thread=8)

conversation_history = []

def store_interaction(user_question, agent_response):
    conversation_history.append({"question": user_question, "response": agent_response})

def get_conversation_context():
    # Concatenate previous Q&A pairs into a single string
    context = ""
    for interaction in conversation_history:
        context += f"Question: {interaction['question']} Answer: {interaction['response']} "
    return context
    

embedding


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

embedded
questions_and_answers.docx has been created successfully.


In [25]:
import os
import time
import pygame
from gtts import gTTS

def play_text_to_speech(text, language='en', slow=False):
    tts = gTTS(text=text, lang=language, slow=slow)
    
    temp_audio_file = "temp_audio.mp3"
    tts.save(temp_audio_file)
    
    pygame.mixer.init()
    pygame.mixer.music.load(temp_audio_file)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

    pygame.mixer.music.stop()
    pygame.mixer.quit()

    time.sleep(3)
    os.remove(temp_audio_file)

In [36]:
pydantic_response = pydantic_llava(
    "llava-13b", Info, ins_image_documents, prompt_template_str
)

for response in pydantic_response:
    print(response)
for question in pydantic_response.question:
    output = customer_agent.query(question)
    if output:
        print(output)
        output = output.lstrip()
        play_text_to_speech(str(output))
        print(f"AI Assistant:{output}")

ReplicateMultiModal currently only supports uploading one image documentwe are using the first image document for completion.


> Raw output: {
"flightNumber": "34258721",
"question": [
"What is my flight information?",
"What is my flight date?",
"Can I change my flight date?"
]
}
('flightNumber', '34258721')
('question', ['What is my flight information?', 'What is my flight date?', 'Can I change my flight date?'])
('flightNumber', '34258721')
('question', ['What is my flight information?', 'What is my flight date?', 'Can I change my flight date?'])


ValueError: Reached max iterations.